In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
from xgboost.sklearn import XGBClassifier

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
import datetime

In [94]:
import lightgbm as lgb

In [5]:
train = pd.read_csv('10_perc_train.csv')
test = pd.read_csv('test.csv')

In [6]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,c,InternetExplorer,NaN,1
1,IDCLNGRY0,2017-01-20 11:23:06,NaN,239178,40339,71040875,b,Edge,Tablet,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,b,Firefox,Mobile,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,f,InternetExplorer,Desktop,0
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,b,InternetExplorer,Desktop,0


In [7]:
train.describe()

,siteid,offerid,category,merchant,click
count,1.092587e+06,1.213781e+06,1.213781e+06,1.213781e+06,1.213781e+06
mean,5.020632e+06,4.965269e+05,4.211022e+04,4.900745e+07,3.605181e-02
std,2.903768e+06,2.899390e+05,2.949769e+04,2.917781e+07,1.864192e-01
min,1.470000e+02,0.000000e+00,9.700000e+01,2.604430e+05,0.000000e+00
25%,2.500598e+06,2.473350e+05,1.496000e+04,2.639177e+07,0.000000e+00
50%,4.970751e+06,4.950460e+05,4.033900e+04,4.861553e+07,0.000000e+00
75%,7.579006e+06,7.477440e+05,6.678600e+04,7.436361e+07,0.000000e+00
max,9.999996e+06,9.999980e+05,9.991400e+04,9.977043e+07,1.000000e+00


In [8]:
train.isnull().sum()

ID                  0
datetime            0
siteid         121194
offerid             0
category            0
merchant            0
countrycode         0
browserid       60749
devid          182070
click               0
dtype: int64

In [9]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'countrycode', 'browserid', 'devid', 'click'],
      dtype='object')

In [10]:
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [11]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,c,InternetExplorer,NaN,1
1,IDCLNGRY0,2017-01-20 11:23:06,NaN,239178,40339,71040875,b,Edge,Tablet,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,b,Firefox,Mobile,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,f,InternetExplorer,Desktop,0
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,b,InternetExplorer,Desktop,0


In [13]:
train['weekday'] = train['datetime'].apply(lambda x:x.dayofweek)
train['hour'] = train['datetime'].apply(lambda x:x.hour)

In [14]:
test['weekday'] = test['datetime'].apply(lambda x:x.dayofweek)
test['hour'] = test['datetime'].apply(lambda x:x.hour)

In [15]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,weekday,hour
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,c,InternetExplorer,NaN,1,4,21
1,IDCLNGRY0,2017-01-20 11:23:06,NaN,239178,40339,71040875,b,Edge,Tablet,0,4,11
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,b,Firefox,Mobile,0,4,12
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,f,InternetExplorer,Desktop,0,2,19
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,b,InternetExplorer,Desktop,0,0,11


In [16]:
dum_week = pd.get_dummies(train['weekday'],prefix='weekday_')
dum_week.head()

,weekday__0,weekday__1,weekday__2,weekday__3,weekday__4,weekday__5,weekday__6
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0


In [17]:
dum_hour = pd.get_dummies(train['hour'],prefix='hour_')
dum_hour.head()

,hour__0,hour__1,hour__2,hour__3,hour__4,hour__5,hour__6,hour__7,hour__8,hour__9,...,hour__14,hour__15,hour__16,hour__17,hour__18,hour__19,hour__20,hour__21,hour__22,hour__23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
train = pd.concat([train,dum_hour,dum_week],axis=1)
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,...,hour__21,hour__22,hour__23,weekday__0,weekday__1,weekday__2,weekday__3,weekday__4,weekday__5,weekday__6
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,c,InternetExplorer,NaN,1,...,1,0,0,0,0,0,0,1,0,0
1,IDCLNGRY0,2017-01-20 11:23:06,NaN,239178,40339,71040875,b,Edge,Tablet,0,...,0,0,0,0,0,0,0,1,0,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,b,Firefox,Mobile,0,...,0,0,0,0,0,0,0,1,0,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,f,InternetExplorer,Desktop,0,...,0,0,0,0,0,1,0,0,0,0
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,b,InternetExplorer,Desktop,0,...,0,0,0,1,0,0,0,0,0,0


In [19]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'countrycode', 'browserid', 'devid', 'click', 'weekday', 'hour',
       'hour__0', 'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5',
       'hour__6', 'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11',
       'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17',
       'hour__18', 'hour__19', 'hour__20', 'hour__21', 'hour__22', 'hour__23',
       'weekday__0', 'weekday__1', 'weekday__2', 'weekday__3', 'weekday__4',
       'weekday__5', 'weekday__6'],
      dtype='object')

In [20]:
train['siteid'] = train['siteid'].fillna(-1)

In [21]:
train['siteid'].isnull().sum()

0

In [22]:
train['offerid'].isnull().sum()

0

In [23]:
train['category'].isnull().sum()

0

In [24]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'countrycode', 'browserid', 'devid', 'click', 'weekday', 'hour',
       'hour__0', 'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5',
       'hour__6', 'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11',
       'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17',
       'hour__18', 'hour__19', 'hour__20', 'hour__21', 'hour__22', 'hour__23',
       'weekday__0', 'weekday__1', 'weekday__2', 'weekday__3', 'weekday__4',
       'weekday__5', 'weekday__6'],
      dtype='object')

In [26]:
train['merchant'].isnull().sum()

0

In [28]:
train['countrycode'].value_counts()

b    529106
a    294740
e    119334
f    118811
c     80358
d     71432
Name: countrycode, dtype: int64

In [29]:
dum_coun = pd.get_dummies(train['countrycode'])
dum_coun.head()

,a,b,c,d,e,f
0,0,0,1,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,0,0,0,0,1
4,0,1,0,0,0,0


In [31]:
train = train.drop(['weekday','hour'],axis=1)
train.head()

,ID,datetime,siteid,offerid,category,merchant,browserid,devid,click,hour__0,...,hour__21,hour__22,hour__23,weekday__0,weekday__1,weekday__2,weekday__3,weekday__4,weekday__5,weekday__6
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,InternetExplorer,NaN,1,0,...,1,0,0,0,0,0,0,1,0,0
1,IDCLNGRY0,2017-01-20 11:23:06,-1.0,239178,40339,71040875,Edge,Tablet,0,0,...,0,0,0,0,0,0,0,1,0,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,Firefox,Mobile,0,0,...,0,0,0,0,0,0,0,1,0,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,InternetExplorer,Desktop,0,0,...,0,0,0,0,0,1,0,0,0,0
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,InternetExplorer,Desktop,0,0,...,0,0,0,1,0,0,0,0,0,0


In [33]:
train = pd.concat([train,dum_coun],axis=1)
train.head()

,ID,datetime,siteid,offerid,category,merchant,browserid,devid,click,hour__0,...,weekday__3,weekday__4,weekday__5,weekday__6,a,b,c,d,e,f
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,InternetExplorer,NaN,1,0,...,0,1,0,0,0,0,1,0,0,0
1,IDCLNGRY0,2017-01-20 11:23:06,-1.0,239178,40339,71040875,Edge,Tablet,0,0,...,0,1,0,0,0,1,0,0,0,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,Firefox,Mobile,0,0,...,0,1,0,0,0,1,0,0,0,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,InternetExplorer,Desktop,0,0,...,0,0,0,0,0,0,0,0,0,1
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,InternetExplorer,Desktop,0,0,...,0,0,0,0,0,1,0,0,0,0


In [34]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'browserid', 'devid', 'click', 'hour__0', 'hour__1', 'hour__2',
       'hour__3', 'hour__4', 'hour__5', 'hour__6', 'hour__7', 'hour__8',
       'hour__9', 'hour__10', 'hour__11', 'hour__12', 'hour__13', 'hour__14',
       'hour__15', 'hour__16', 'hour__17', 'hour__18', 'hour__19', 'hour__20',
       'hour__21', 'hour__22', 'hour__23', 'weekday__0', 'weekday__1',
       'weekday__2', 'weekday__3', 'weekday__4', 'weekday__5', 'weekday__6',
       'a', 'b', 'c', 'd', 'e', 'f'],
      dtype='object')

In [36]:
train['browserid'].value_counts()

Edge                 345873
Firefox              334829
Mozilla              111869
Mozilla Firefox      100978
InternetExplorer      74501
Google Chrome         69567
IE                    34647
Chrome                34486
Internet Explorer     23247
Safari                11521
Opera                 11514
Name: browserid, dtype: int64

In [38]:
firefox_list = ['Firefox','Mozilla','Mozilla Firefox']
chrome_list = ['Google Chrome','Chrome']
ie_list = ['IE','InternetExplorer','Internet Explorer']

In [39]:
def update_browser_id(x):
    if x in firefox_list:
        return 'ff'
    if x in chrome_list:
        return 'gc'
    if x in ie_list:
        return 'ie'
    return x

In [41]:
train['browserid'] = train['browserid'].apply(lambda x:update_browser_id(x))

In [45]:
train['browserid'].value_counts()

ff        608425
Edge      345873
ie        132395
gc        104053
Safari     11521
Opera      11514
Name: browserid, dtype: int64

In [44]:
train['browserid'] = train['browserid'].fillna('ff')
train['browserid'].isnull().sum()

0

In [46]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'browserid', 'devid', 'click', 'hour__0', 'hour__1', 'hour__2',
       'hour__3', 'hour__4', 'hour__5', 'hour__6', 'hour__7', 'hour__8',
       'hour__9', 'hour__10', 'hour__11', 'hour__12', 'hour__13', 'hour__14',
       'hour__15', 'hour__16', 'hour__17', 'hour__18', 'hour__19', 'hour__20',
       'hour__21', 'hour__22', 'hour__23', 'weekday__0', 'weekday__1',
       'weekday__2', 'weekday__3', 'weekday__4', 'weekday__5', 'weekday__6',
       'a', 'b', 'c', 'd', 'e', 'f'],
      dtype='object')

In [47]:
dum_browser = pd.get_dummies(train['browserid'],prefix='brow_')
dum_browser.head()

,brow__Edge,brow__Opera,brow__Safari,brow__ff,brow__gc,brow__ie
0,0,0,0,0,0,1
1,1,0,0,0,0,0
2,0,0,0,1,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [48]:
train = train.drop(['browserid'],axis=1)
train.head()

,ID,datetime,siteid,offerid,category,merchant,devid,click,hour__0,hour__1,...,weekday__3,weekday__4,weekday__5,weekday__6,a,b,c,d,e,f
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,NaN,1,0,0,...,0,1,0,0,0,0,1,0,0,0
1,IDCLNGRY0,2017-01-20 11:23:06,-1.0,239178,40339,71040875,Tablet,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,Mobile,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,Desktop,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,Desktop,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [49]:
train = pd.concat([train,dum_browser],axis=1)
train.head()

,ID,datetime,siteid,offerid,category,merchant,devid,click,hour__0,hour__1,...,c,d,e,f,brow__Edge,brow__Opera,brow__Safari,brow__ff,brow__gc,brow__ie
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,NaN,1,0,0,...,1,0,0,0,0,0,0,0,0,1
1,IDCLNGRY0,2017-01-20 11:23:06,-1.0,239178,40339,71040875,Tablet,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,Mobile,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,Desktop,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,Desktop,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [50]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant', 'devid',
       'click', 'hour__0', 'hour__1', 'hour__2', 'hour__3', 'hour__4',
       'hour__5', 'hour__6', 'hour__7', 'hour__8', 'hour__9', 'hour__10',
       'hour__11', 'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16',
       'hour__17', 'hour__18', 'hour__19', 'hour__20', 'hour__21', 'hour__22',
       'hour__23', 'weekday__0', 'weekday__1', 'weekday__2', 'weekday__3',
       'weekday__4', 'weekday__5', 'weekday__6', 'a', 'b', 'c', 'd', 'e', 'f',
       'brow__Edge', 'brow__Opera', 'brow__Safari', 'brow__ff', 'brow__gc',
       'brow__ie'],
      dtype='object')

In [52]:
train['devid'].value_counts()

Mobile     403305
Tablet     340856
Desktop    287550
Name: devid, dtype: int64

In [55]:
train['devid'].isnull().sum()

0

In [54]:
train['devid'] = train['devid'].fillna('Mobile')

In [56]:
dum_device = pd.get_dummies(train['devid'])
dum_device.head()

,Desktop,Mobile,Tablet
0,0,1,0
1,0,0,1
2,0,1,0
3,1,0,0
4,1,0,0


In [ ]:
# create aggregate features
site_offer_count = train.groupby(['siteid','offerid']).size().reset_index()
site_offer_count.columns = ['siteid','offerid','site_offer_count']

site_offer_count_test = test.groupby(['siteid','offerid']).size().reset_index()
site_offer_count_test.columns = ['siteid','offerid','site_offer_count']

site_cat_count = train.groupby(['siteid','category']).size().reset_index()
site_cat_count.columns = ['siteid','category','site_cat_count']

site_cat_count_test = test.groupby(['siteid','category']).size().reset_index()
site_cat_count_test.columns = ['siteid','category','site_cat_count']

site_mcht_count = train.groupby(['siteid','merchant']).size().reset_index()
site_mcht_count.columns = ['siteid','merchant','site_mcht_count']

In [ ]:
# joining all files
agg_df = [site_offer_count,site_cat_count,site_mcht_count]
agg_df_test = [site_offer_count_test,site_cat_count_test,site_mcht_count_test]

for x in agg_df:
    train = train.merge(x)
    
for x in agg_df_test:
    test = test.merge(x)

In [57]:
train = train.drop(['devid'],axis=1)
train = pd.concat([train,dum_device],axis=1)
train.head()

,ID,datetime,siteid,offerid,category,merchant,click,hour__0,hour__1,hour__2,...,f,brow__Edge,brow__Opera,brow__Safari,brow__ff,brow__gc,brow__ie,Desktop,Mobile,Tablet
0,IDMUUBNwz,2017-01-20 21:48:49,7615567.0,138618,11837,79474990,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,IDCLNGRY0,2017-01-20 11:23:06,-1.0,239178,40339,71040875,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,IDnxGiOqx,2017-01-13 12:44:48,1512420.0,185903,34670,48615529,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,IDV8WvGu0,2017-01-11 19:47:32,6389661.0,393187,42227,30860214,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,IDQVAl6RK,2017-01-16 11:54:41,2352540.0,117949,14960,555603,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [64]:
train.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant', 'click',
       'hour__0', 'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5',
       'hour__6', 'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11',
       'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17',
       'hour__18', 'hour__19', 'hour__20', 'hour__21', 'hour__22', 'hour__23',
       'weekday__0', 'weekday__1', 'weekday__2', 'weekday__3', 'weekday__4',
       'weekday__5', 'weekday__6', 'a', 'b', 'c', 'd', 'e', 'f', 'brow__Edge',
       'brow__Opera', 'brow__Safari', 'brow__ff', 'brow__gc', 'brow__ie',
       'Desktop', 'Mobile', 'Tablet'],
      dtype='object')

In [58]:
test.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'countrycode', 'browserid', 'devid', 'weekday', 'hour'],
      dtype='object')

In [60]:
dum_week = pd.get_dummies(test['weekday'],prefix='weekday_')
dum_hour = pd.get_dummies(test['hour'],prefix='hour_')
test['siteid'] = test['siteid'].fillna(-1)
dum_coun = pd.get_dummies(test['countrycode'])
test['browserid'] = test['browserid'].apply(lambda x:update_browser_id(x))
test['browserid'] = test['browserid'].fillna('ff')
dum_browser = pd.get_dummies(test['browserid'],prefix='brow_')
test['devid'] = test['devid'].fillna('Mobile')
dum_device = pd.get_dummies(test['devid'])
test = test.drop(['weekday','hour','countrycode','browserid','devid'],axis=1)
test = pd.concat([test,dum_coun,dum_week,dum_hour,dum_browser,dum_device],axis=1)

In [69]:
#train.to_csv('updated_train.csv',index=False)
test.to_csv('updated_test.csv',index=False)

In [68]:
test.head()

,ID,datetime,siteid,offerid,category,merchant,a,b,c,d,...,brow__ff,brow__gc,brow__ie,Desktop,Mobile,Tablet,weekday_1,weekday_2,weekday_3,weekday_4
0,IDFDJVI,2017-01-22 09:55:48,755610.0,808980,17714,26391770,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
1,IDNWkTQ,2017-01-22 03:54:39,3714899.0,280355,12052,39507200,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ID9pRmM,2017-01-21 10:25:50,4378333.0,930819,30580,46148550,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,IDHaQaj,2017-01-22 14:45:53,1754730.0,612234,11837,8837581,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,IDT2CrF,2017-01-22 09:34:07,5299909.0,524289,45620,31388981,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [66]:
for i in range(1,5):
    test['weekday_'+str(i)]=0

In [74]:
test.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant', 'a', 'b',
       'c', 'd', 'e', 'f', 'weekday__0', 'weekday__5', 'weekday__6', 'hour__0',
       'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5', 'hour__6',
       'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11', 'hour__12',
       'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17', 'hour__18',
       'hour__19', 'hour__20', 'hour__21', 'hour__22', 'hour__23',
       'brow__Edge', 'brow__Opera', 'brow__Safari', 'brow__ff', 'brow__gc',
       'brow__ie', 'Desktop', 'Mobile', 'Tablet', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4'],
      dtype='object')

In [76]:
test.head()

,ID,datetime,siteid,offerid,category,merchant,a,b,c,d,...,brow__ff,brow__gc,brow__ie,Desktop,Mobile,Tablet,weekday_1,weekday_2,weekday_3,weekday_4
0,IDFDJVI,2017-01-22 09:55:48,755610.0,808980,17714,26391770,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
1,IDNWkTQ,2017-01-22 03:54:39,3714899.0,280355,12052,39507200,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ID9pRmM,2017-01-21 10:25:50,4378333.0,930819,30580,46148550,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,IDHaQaj,2017-01-22 14:45:53,1754730.0,612234,11837,8837581,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,IDT2CrF,2017-01-22 09:34:07,5299909.0,524289,45620,31388981,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [77]:
test.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant', 'a', 'b',
       'c', 'd', 'e', 'f', 'weekday__0', 'weekday__5', 'weekday__6', 'hour__0',
       'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5', 'hour__6',
       'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11', 'hour__12',
       'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17', 'hour__18',
       'hour__19', 'hour__20', 'hour__21', 'hour__22', 'hour__23',
       'brow__Edge', 'brow__Opera', 'brow__Safari', 'brow__ff', 'brow__gc',
       'brow__ie', 'Desktop', 'Mobile', 'Tablet', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4'],
      dtype='object')

In [70]:
xgb = XGBClassifier(learning_rate =0.01, n_estimators=1000, max_depth=6,
                    min_child_weight=7, gamma=0.01, subsample=0.8, colsample_bytree=0.8,
                    objective= 'binary:logistic', nthread=4, seed=2017)

In [88]:
features = ['siteid', 'offerid', 'category', 'merchant', 'a', 'b',
       'c', 'd', 'e', 'f', 'weekday__0', 'weekday__5', 'weekday__6', 'hour__0',
       'hour__1', 'hour__2', 'hour__3', 'hour__4', 'hour__5', 'hour__6',
       'hour__7', 'hour__8', 'hour__9', 'hour__10', 'hour__11', 'hour__12',
       'hour__13', 'hour__14', 'hour__15', 'hour__16', 'hour__17', 'hour__18',
       'hour__19', 'hour__20', 'hour__21', 'hour__22', 'hour__23',
       'brow__Edge', 'brow__Opera', 'brow__Safari', 'brow__ff', 'brow__gc',
       'brow__ie', 'Desktop', 'Mobile', 'Tablet']

In [ ]:
xgb.fit(train[features],train['click'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.01, learning_rate=0.01,
       max_delta_step=0, max_depth=6, min_child_weight=7, missing=None,
       n_estimators=1000, nthread=4, objective='binary:logistic',
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2017,
       silent=True, subsample=0.8)

In [ ]:
pred = xgb.predict_proba(test[features])

In [ ]:
sub = pd.DataFrame({'ID':test['ID'], 'click':pred[:,1]})
sub.to_csv('xgb.csv', index=False)

In [93]:
print(len(sub))

3706907


In [95]:
dtrain = lgb.Dataset(train[features],train['click'])

In [96]:
params = {
    'num_leaves' : 256,
    'learning_rate':0.03,
    'metric':'auc',
    'objective':'binary',
    'early_stopping_round': 40,
    'max_depth':10,
    'bagging_fraction':0.5,
    'feature_fraction':0.6,
    'bagging_seed':2017,
    'feature_fraction_seed':2017,
    'verbose' : 1   
}

In [ ]:
lgbm =lgb.train(params, dtrain,num_boost_round=500,verbose_eval=20)

In [ ]:
pred_lgb = lgbm.predict(test[features])

In [103]:
sub = pd.DataFrame({'ID':test['ID'], 'click':pred_lgb})
sub.to_csv('lgb.csv', index=False)

In [104]:
from catboost import CatBoostClassifier

In [105]:
cb = CatBoostClassifier(depth=10, iterations=10, learning_rate=0.1, eval_metric='AUC', random_seed=1)

In [107]:
cb.fit(train[features],train['click'])

In [108]:
pred_cb = cb.predict_proba(test[features])
sub = pd.DataFrame({'ID':test['ID'], 'click':pred_cb[:,1]})
sub.to_csv('cb.csv', index=False)

In [15]:
rfc = RandomForestClassifier()

In [16]:
rfc.fit(train[features],train['click'])
pred = rfc.predict_proba(test[features])
sub = pd.DataFrame({'ID':test['ID'], 'click':pred[:,1]})
sub.to_csv('rfc.csv', index=False)